In [1]:
import torchtext
import torch
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from torchtext.vocab import Vectors
from tqdm import tqdm_notebook
import pandas as pd

In [2]:
train_df = pd.read_csv("/kaggle/input/sst-2-dataset/train.csv")
validation_df = pd.read_csv("/kaggle/input/sst-2-dataset/validation.csv")

train_df.drop(columns = "idx")
validation_df.drop(columns = "idx")


test_df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
test_df.rename(columns={'review': 'sentence', 'sentiment': 'label'}, inplace=True)
test_df['label'] = test_df['label'].map({'negative': 0, 'positive': 1})
output_path = "/kaggle/working/modified_imdb_dataset.csv"
test_df.to_csv(output_path, index=False)

In [3]:
train_df["sentence"] = train_df["sentence"].str.lower()
validation_df["sentence"] = validation_df["sentence"].str.lower()
test_df["sentence"] = test_df["sentence"].str.lower()

In [4]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

train_df["text_wo_punct"] = train_df["sentence"].apply(lambda text: remove_punctuation(text))
validation_df["text_wo_punct"] = validation_df["sentence"].apply(lambda text: remove_punctuation(text))
test_df["text_wo_punct"] = test_df["sentence"].apply(lambda text: remove_punctuation(text))

In [5]:
import nltk

# Ensure you have downloaded the stopwords dataset
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords

", ".join(stopwords.words('english'))


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [7]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

train_df["text_wo_stop"] = train_df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
validation_df["text_wo_stop"] = validation_df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
test_df["text_wo_stop"] = test_df["text_wo_punct"].apply(lambda text: remove_stopwords(text))

In [8]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

train_df["text_stemmed"] = train_df["text_wo_stop"].apply(lambda text: stem_words(text))
validation_df["text_stemmed"] = validation_df["text_wo_stop"].apply(lambda text: stem_words(text))
test_df["text_stemmed"] = test_df["text_wo_stop"].apply(lambda text: stem_words(text))

In [9]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize


# Tokenize sentences
nltk.download('punkt')
train_df['tokens'] = train_df['text_stemmed'].apply(lambda x: nltk.word_tokenize(x))

# Create a word index (manual embedding)
word_set = set()
for tokens in train_df['tokens']:
    word_set.update(tokens)

word_index = {word: i+1 for i, word in enumerate(word_set)}  # Start indexing from 1

# Convert tokens to integers
train_df['indexed_tokens'] = train_df['tokens'].apply(lambda x: [word_index[token] for token in x])

# Optional: Pad sequences to ensure uniform length
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = 100  # Define maximum length of sequences
train_df['padded_tokens'] = pad_sequences(train_df['indexed_tokens'], maxlen=max_length, padding='post').tolist()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2024-05-29 15:18:45.125166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 15:18:45.125277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 15:18:45.269495: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
train_df

,idx,sentence,label,text_wo_punct,text_wo_stop,text_stemmed,tokens,indexed_tokens,padded_tokens
0,0,hide new secretions from the parental units,0,hide new secretions from the parental units,hide new secretions parental units,hide new secret parent unit,"[hide, new, secret, parent, unit]","[1468, 9934, 2775, 1438, 2739]","[1468, 9934, 2775, 1438, 2739, 0, 0, 0, 0, 0, ..."
1,1,"contains no wit , only labored gags",0,contains no wit only labored gags,contains wit labored gags,contain wit labor gag,"[contain, wit, labor, gag]","[1113, 10676, 422, 2198]","[1113, 10676, 422, 2198, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,that loves its characters and communicates som...,1,that loves its characters and communicates som...,loves characters communicates something rather...,love charact commun someth rather beauti human...,"[love, charact, commun, someth, rather, beauti...","[1681, 6313, 7531, 4957, 2439, 1141, 2217, 2471]","[1681, 6313, 7531, 4957, 2439, 1141, 2217, 247..."
3,3,remains utterly satisfied to remain the same t...,0,remains utterly satisfied to remain the same t...,remains utterly satisfied remain throughout,remain utterli satisfi remain throughout,"[remain, utterli, satisfi, remain, throughout]","[6225, 5358, 7088, 6225, 9407]","[6225, 5358, 7088, 6225, 9407, 0, 0, 0, 0, 0, ..."
4,4,on the worst revenge-of-the-nerds clichés the ...,0,on the worst revengeofthenerds clichés the fil...,worst revengeofthenerds clichés filmmakers cou...,worst revengeofthenerd cliché filmmak could dredg,"[worst, revengeofthenerd, cliché, filmmak, cou...","[3647, 5690, 8997, 1136, 10290, 7921]","[3647, 5690, 8997, 1136, 10290, 7921, 0, 0, 0,..."
...,...,...,...,...,...,...,...,...,...
67344,67344,a delightful comedy,1,a delightful comedy,delightful comedy,delight comedi,"[delight, comedi]","[7309, 8380]","[7309, 8380, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
67345,67345,"anguish , anger and frustration",0,anguish anger and frustration,anguish anger frustration,anguish anger frustrat,"[anguish, anger, frustrat]","[3588, 7222, 4072]","[3588, 7222, 4072, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
67346,67346,"at achieving the modest , crowd-pleasing goals...",1,at achieving the modest crowdpleasing goals i...,achieving modest crowdpleasing goals sets,achiev modest crowdpleas goal set,"[achiev, modest, crowdpleas, goal, set]","[9720, 3480, 9079, 1085, 6819]","[9720, 3480, 9079, 1085, 6819, 0, 0, 0, 0, 0, ..."
67347,67347,a patient viewer,1,a patient viewer,patient viewer,patient viewer,"[patient, viewer]","[206, 4606]","[206, 4606, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."


In [11]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize


# Tokenize sentences
nltk.download('punkt')
validation_df['tokens'] = validation_df['text_stemmed'].apply(lambda x: nltk.word_tokenize(x))

# Create a word index (manual embedding)
word_set = set()
for tokens in validation_df['tokens']:
    word_set.update(tokens)

word_index = {word: i+1 for i, word in enumerate(word_set)}  # Start indexing from 1

# Convert tokens to integers
validation_df['indexed_tokens'] = validation_df['tokens'].apply(lambda x: [word_index[token] for token in x])

# Optional: Pad sequences to ensure uniform length
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = 100  # Define maximum length of sequences
validation_df['padded_tokens'] = pad_sequences(validation_df['indexed_tokens'], maxlen=max_length, padding='post').tolist()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
validation_df

,idx,sentence,label,text_wo_punct,text_wo_stop,text_stemmed,tokens,indexed_tokens,padded_tokens
0,0,it 's a charming and often affecting journey .,1,it s a charming and often affecting journey,charming often affecting journey,charm often affect journey,"[charm, often, affect, journey]","[530, 2220, 2475, 1829]","[530, 2220, 2475, 1829, 0, 0, 0, 0, 0, 0, 0, 0..."
1,1,unflinchingly bleak and desperate,0,unflinchingly bleak and desperate,unflinchingly bleak desperate,unflinchingli bleak desper,"[unflinchingli, bleak, desper]","[1112, 10, 333]","[1112, 10, 333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,allows us to hope that nolan is poised to emba...,1,allows us to hope that nolan is poised to emba...,allows us hope nolan poised embark major caree...,allow us hope nolan pois embark major career c...,"[allow, us, hope, nolan, pois, embark, major, ...","[343, 2803, 2841, 1977, 770, 2604, 1856, 2965,...","[343, 2803, 2841, 1977, 770, 2604, 1856, 2965,..."
3,3,"the acting , costumes , music , cinematography...",1,the acting costumes music cinematography an...,acting costumes music cinematography sound ast...,act costum music cinematographi sound astound ...,"[act, costum, music, cinematographi, sound, as...","[1924, 2564, 215, 1859, 2470, 2892, 1024, 702,...","[1924, 2564, 215, 1859, 2470, 2892, 1024, 702,..."
4,4,"it 's slow -- very , very slow .",0,it s slow very very slow,slow slow,slow slow,"[slow, slow]","[747, 747]","[747, 747, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...,...,...,...
867,867,has all the depth of a wading pool .,0,has all the depth of a wading pool,depth wading pool,depth wade pool,"[depth, wade, pool]","[407, 905, 520]","[407, 905, 520, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
868,868,a movie with a real anarchic flair .,1,a movie with a real anarchic flair,movie real anarchic flair,movi real anarch flair,"[movi, real, anarch, flair]","[848, 575, 1572, 2426]","[848, 575, 1572, 2426, 0, 0, 0, 0, 0, 0, 0, 0,..."
869,869,a subject like this should inspire reaction in...,0,a subject like this should inspire reaction in...,subject like inspire reaction audience pianist,subject like inspir reaction audienc pianist,"[subject, like, inspir, reaction, audienc, pia...","[370, 3375, 536, 1860, 1355, 257]","[370, 3375, 536, 1860, 1355, 257, 0, 0, 0, 0, ..."
870,870,... is an arthritic attempt at directing by ca...,0,is an arthritic attempt at directing by calli...,arthritic attempt directing callie khouri,arthrit attempt direct calli khouri,"[arthrit, attempt, direct, calli, khouri]","[2123, 2566, 1550, 1684, 622]","[2123, 2566, 1550, 1684, 622, 0, 0, 0, 0, 0, 0..."


In [13]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize


# Tokenize sentences
nltk.download('punkt')

test_df['tokens'] = test_df['text_stemmed'].apply(lambda x: nltk.word_tokenize(x))

# Create a word index (manual embedding)
word_set = set()
for tokens in test_df['tokens']:
    word_set.update(tokens)

word_index = {word: i+1 for i, word in enumerate(word_set)}  # Start indexing from 1

# Convert tokens to integers
test_df['indexed_tokens'] = test_df['tokens'].apply(lambda x: [word_index[token] for token in x])

# Optional: Pad sequences to ensure uniform length
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = 100  # Define maximum length of sequences
test_df['padded_tokens'] = pad_sequences(test_df['indexed_tokens'], maxlen=max_length, padding='post').tolist()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
test_df.head()

,sentence,label,text_wo_punct,text_wo_stop,text_stemmed,tokens,indexed_tokens,padded_tokens
0,one of the other reviewers has mentioned that ...,1,one of the other reviewers has mentioned that ...,one reviewers mentioned watching 1 oz episode ...,one review mention watch 1 oz episod youll hoo...,"[one, review, mention, watch, 1, oz, episod, y...","[40808, 108034, 54983, 140673, 125030, 113018,...","[39483, 59454, 115822, 31880, 74736, 118890, 5..."
1,a wonderful little production. <br /><br />the...,1,a wonderful little production br br the filmin...,wonderful little production br br filming tech...,wonder littl product br br film techniqu unass...,"[wonder, littl, product, br, br, film, techniq...","[13202, 68820, 103460, 87348, 87348, 104296, 2...","[13202, 68820, 103460, 87348, 87348, 104296, 2..."
2,i thought this was a wonderful way to spend ti...,1,i thought this was a wonderful way to spend ti...,thought wonderful way spend time hot summer we...,thought wonder way spend time hot summer weeke...,"[thought, wonder, way, spend, time, hot, summe...","[41312, 13202, 134697, 33439, 62867, 111511, 6...","[41312, 13202, 134697, 33439, 62867, 111511, 6..."
3,basically there's a family where a little boy ...,0,basically theres a family where a little boy j...,basically theres family little boy jake thinks...,basic there famili littl boy jake think there ...,"[basic, there, famili, littl, boy, jake, think...","[35956, 141474, 97235, 68820, 37102, 80826, 30...","[35956, 141474, 97235, 68820, 37102, 80826, 30..."
4,"petter mattei's ""love in the time of money"" is...",1,petter matteis love in the time of money is a ...,petter matteis love time money visually stunni...,petter mattei love time money visual stun film...,"[petter, mattei, love, time, money, visual, st...","[137493, 38676, 38617, 62867, 94959, 77598, 73...","[59242, 132649, 33133, 131046, 91807, 33380, 1..."


In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder


param_grid = {
    'tfidf__max_df': [0.95],
    'tfidf__ngram_range': [(1, 3)],
    'rf__n_estimators': [200],
    'rf__max_depth': [None],
    'rf__criterion': ['entropy'],
    'rf__min_samples_leaf': [2]
}


X_train = train_df['text_wo_stop']
y_train = train_df['label']
X_valid = validation_df['text_wo_stop']
y_valid = validation_df['label']


# Create the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestClassifier(random_state=42))
])

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=2, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters and estimator
print("Best parameters found: ", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Evaluate on validation set
y_valid_pred = best_model.predict(X_valid)
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
valid_report = classification_report(y_valid, y_valid_pred)

print(f"Validation Accuracy: {valid_accuracy}")
print("Validation Classification Report:")
print(valid_report)


Fitting 2 folds for each of 1 candidates, totalling 2 fits
Best parameters found:  {'rf__criterion': 'entropy', 'rf__max_depth': None, 'rf__min_samples_leaf': 2, 'rf__n_estimators': 200, 'tfidf__max_df': 0.95, 'tfidf__ngram_range': (1, 3)}
Validation Accuracy: 0.7786697247706422
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.71      0.76       428
           1       0.75      0.84      0.79       444

    accuracy                           0.78       872
   macro avg       0.78      0.78      0.78       872
weighted avg       0.78      0.78      0.78       872



In [22]:
X_test = test_df['text_wo_stop']
y_test = test_df['label']

y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_report = classification_report(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy}")
print("Test Classification Report:")
print(test_report)

[CV] END rf__criterion=entropy, rf__max_depth=None, rf__min_samples_leaf=2, rf__n_estimators=200, tfidf__max_df=0.95, tfidf__ngram_range=(1, 3); total time= 1.9min
[CV] END rf__criterion=entropy, rf__max_depth=None, rf__min_samples_leaf=2, rf__n_estimators=200, tfidf__max_df=0.95, tfidf__ngram_range=(1, 3); total time= 1.9min
Test Accuracy: 0.79592
Test Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.86      0.81     25000
           1       0.84      0.73      0.78     25000

    accuracy                           0.80     50000
   macro avg       0.80      0.80      0.80     50000
weighted avg       0.80      0.80      0.80     50000

